In [1]:
import tailer
import re
import threading
import csv

def monitor_log_file(log_file_path, patterns, alert_message):
    """
    Monitor a log file for new entries matching any of the specified patterns and alert when such an entry is found.
    Prints the matching string along with the alert message.
    """
    compiled_patterns = [re.compile(pattern) for pattern in patterns]
    with open(log_file_path, "r") as file:
        for line in tailer.follow(file):
            for compiled_pattern in compiled_patterns:
                match = compiled_pattern.search(line)
                if match:
                    matching_string = match.group()
                    print(f"{alert_message} {matching_string}")
                    break  # Stop checking patterns after the first match

def monitor_csv_file(csv_file_path, patterns, alert_message, column_index=0):
    compiled_patterns = [re.compile(pattern) for pattern in patterns]
    with open(csv_file_path, "r") as file:
        for line in tailer.follow(file):
            reader = csv.reader([line])
            for row in reader:
                if len(row) > column_index:
                    for compiled_pattern in compiled_patterns:
                        match = compiled_pattern.search(row[column_index])
                        if match:
                            # Use the original string for re.search
                            unit_pattern = r" LOC_UNIT_([A-Z]*)_NAME"
                            matching_string = re.search(unit_pattern, row[column_index])  # Use the original string here
                            if matching_string:  # Check if the search was successful
                                unit_string = matching_string.group(1)  # Extract the matched group
                                print(f"{alert_message} {unit_string}")
                                break  # Stop checking patterns after the first match

def start_monitoring(files_info):
    """
    Start monitoring multiple files in separate threads, allowing for multiple patterns per file.
    Prints the specific string that triggered the alert.
    """
    threads = []

    for file_info in files_info:
        file_type = file_info.get('type')
        if file_type == 'log':
            thread = threading.Thread(target=monitor_log_file, args=(file_info['path'], file_info['pattern'], file_info['message']))
        elif file_type == 'csv':
            thread = threading.Thread(target=monitor_csv_file, args=(file_info['path'], file_info['pattern'], file_info['message'], file_info.get('column_index', 0)))
        else:
            continue  # Unsupported file type
        
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

# Example usage
files_info = [
    {'type': 'log', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\GameCore.log', 'pattern': r"ERROR", 'message': "Error found"},
    {'type': 'csv', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\Barbarians_Units.csv', 'pattern': [r" LOC_UNIT_SPEARMAN_NAME",
                                                                                                                                                     r" LOC_UNIT_SCOUT_NAME",
                                                                                                                                                       r" LOC_UNIT_WARRIOR_NAME",
                                                                                                                                                         r" LOC_UNIT_BARBARIAN_HORSEMAN_NAME",
                                                                                                                                                         r" LOC_UNIT_GALLEY_NAME",
                                                                                                                                                         r" LOC_UNIT_QUADRIREME_NAME",
                                                                                                                                                         r" LOC_UNIT_SWORDSMAN_NAME",
                                                                                                                                                         r" LOC_UNIT_ARCHER_NAME",
                                                                                                                                                         r" LOC_UNIT_MAN_AT_ARMS_NAME",
                                                                                                                                                         r" LOC_UNIT_CROSSBOWMAN_NAME",
                                                                                                                                                         ], 'message': "New Barbarian Unit spawned:", 'column_index': 2}
]

start_monitoring(files_info)


New Barbarian Unit spawned: SPEARMAN
